In [ ]:
import numpy as np
PI = np.pi
import matplotlib.pyplot as plt
import scipy.special as sp
import math
from scipy import signal
import cmath
from scipy.optimize import curve_fit
import matplotlib.backends.backend_pdf
import os
import pandas as pd


############## Parameters to change #####################
input_file = 'fid' # k-space data

# Acquisition data
xres_ro = 128 # total readout points stored
views = 403 # number of radial views
# angl = (math.sqrt(5)-1)/2 * PI # golden angle in radians
angl = 2*PI/views # used by Steve Pickup

# Image data
xres = 96   # actual number of points collected
yres = 96
slices = 16 
bvalues = 5 
b_array = np.array([10, 535, 1070, 1479, 2141]) # b-values

###################### Run Code ###########################
image_dimensions = [slices, bvalues, yres, xres]
kspace_dimensions = [xres_ro, views, angl]

%run DWI_processing_functions.py

In [ ]:
# Create and save dwis
dwis = reconstruct_radial_DWIs(input_file=input_file, img_dim = image_dimensions, kspace_dim = kspace_dimensions) # Saves DWIs

In [ ]:
SNR_threshold = 5 # minimum SNR to be identified as not background; will not fit voxels below this SNR, speeding up processing
region = [0,10,0,10] # Where "noise" is defined [x1,x2,y1,y2]; by default, top left corner

dwi_fits = diffusion_fit(b_array=b_array, dwis=dwis, SNR_threshold=SNR_threshold, noise_region=region)

In [ ]:
mask_path = "mask.raw"
mask_index = ['Phantom 1', 'Phantom 2', 'Tumor', 'Muscle', 'Kidney']

ROI_results = ROI_analysis(mask_file=mask_path, img_dims=image_dimensions, index=mask_index, dwi_fits=dwi_fits)
ROI_ADC_masks = ROI_results[0]
ROI_ADC_means = ROI_results[1]
ROI_KI_masks = ROI_results[2]
ROI_KI_means = ROI_results[3]